In [2]:
import numpy as np
import pandas as pd
import time, gc
 
pi = np.pi 
cos = np.cos 
sin = np.sin
acos = np.arccos
degrees = np.degrees
radians = np.radians

In [120]:
CATALOGUE = pd.read_csv(r"F:\IIT Bombay\SatLab\Star Tracker\Programs\Catalogues\Modified Star Catalogue.csv")
# StarID: The database primary key from a larger "master database" of stars
# Mag: The star's apparent visual magnitude
# RA, Dec: The star's right ascension and declination, for epoch 2000.0 (Unit: RA - hrs; Dec - degrees)

# Sorts <CATALOGUE>
CATALOGUE.sort_values('Mag', inplace=True)

In [3]:
def angularDistance(row, col_names):
    '''
    Computes the angular distance (in degrees) between two points on the celestial 
    sphere with a given right-ascension and declination values
    
    <Formula> - http://spiff.rit.edu/classes/phys373/lectures/radec/radec.html
    
    Parameters
    ----------
    row : pd.Dataframe - series
        Input right-ascension in (hrs) and declination in (degrees) format
        
    col_names: list of strings
        The names of the columns on which the function will be applied 
        ### SHOULD FOLLOW THIS CONVENTION
        c1 = right-ascension_1; c2 = right-ascension_2
        c3 = declination_1; c4 = declination_2
          
    Returns
    -------
    y : pd.Dataframe - series
        The corresponding angular distance in degree value.
    '''
    # Unpack column names
    c1, c2, c3, c4 = col_names
    # Assert datatypes
    assert type(c1) == str and type(c2) == str and type(c3) == str and type(c4) == str, 'TypeError: input should be str'
      
    
    # Units of right-ascension is in (hours) format
    alpha1, alpha2 = radians(15*row[c1]), radians(15*row[c2])
    
    # Units of declination is in (degrees) format
    delta1, delta2 = radians(row[c3]), radians(row[c4])
    
    # Given Formula
    temp = cos(pi/2 - delta1)*cos(pi/2 - delta2) + sin(pi/2 - delta1)*sin(pi/2 - delta2)*cos(alpha1 - alpha2) 
    
    return np.degrees(acos(temp))

In [153]:
def genSigmaCatalogue(CATALOGUE, mag_limit = 6, FOV_limit = 20):
    '''
    Generates the mean of the sigma for each star in the catalogue.
    
    Sigma between star A and star B is defined as (1/6) of the angular 
    distance between the two stars.
    
    Such values of sigma is calculated for star A to every other star 
    in the catalogue that are its nearest neighbours, i.e., all those
    stars within a circular FOV defined by FOV_limit.
    This is set of sigma values is defined as sigma_n.
    
    The mean of all the elements of sigma_n gives us mu_n.
    This mean value is paired with the corresponding star A.
    
    This process repeats for every star in the catalogue, and the star IDs
    the corresponding mu_n values are collated in a dataframe.
    
    
    Parameters
    ----------
    CATALOGUE : pd.Dataframe
        The 'master' star catalogue on which the function works

    mag_limit : floating-point number
        The upper magnitude limit of stars that are required in the reference catalogue
        
    FOV_limit: floating-point number
        Defines the circular radius which demarcates which stars from the catalogue are 
        considered as nearest neighbours for a given star
        
    Returns
    -------
    SIGMA_CATALOGUE : pd.Dataframe
        The dataframe collated from the star IDs and their corresponding mu_n
    '''
    
    # Start clock-1
    start1 = time.time()
    
    # Generate restricted catalogue based on upper magnitude limit
    temp0 = CATALOGUE[CATALOGUE.Mag <= mag_limit]
    
    # Number of rows in the resticted catalogue
    rows = temp0.shape[0]
    # Resets the index of <temp0>
    temp0.index = list(range(rows))
    
    # Prints total number of stars in <temp0> and the (n)X(n-1)- unique combinations per star
    print('Number of stars - ', rows)
    print('Number of unique combinations per star= ', (rows-1)*rows)
    
    # Initialize the number of iterations to take place
    no_iter = (rows) 
    
    # Initialize SIGMA_CATALOGUE
    SIGMA_CATALOGUE = pd.DataFrame(columns=['Star_ID', 'mu_n'])
    
    for i in range(no_iter):
        # Throws error if an iteration runs beyond number of available rows in <temp0>
        assert i<(rows), 'IndexError: iterating beyond available number of rows'
        
        # Generates <temp1> dataframe which has the (i - th) star of <temp0>
        # repetated (rows-1) times 
        temp1 = pd.DataFrame(columns = ['Star_ID1','RA_1', 'Dec_1', 'Mag_1'])
        s1, ra, dec, mag = temp0.iloc[i]
        temp1.loc[0] = [s1] + [ra] + [dec] + [mag]
        temp1 = pd.concat([temp1]*(rows-1), ignore_index=True)

        # Stores value of the star_ID for which mu_n will be calculated
        star_id_i = s1

        # Generates <temp2> dataframe by copying values of <temp0> and dropping the
        # (i -th) row from it
        temp2 = temp0
        temp2 = temp2.drop([i], axis = 0)
        # Resets the index 
        temp2.index = list(range(0, rows-1))

        # Concatenates <temp1> & <temp2> side-by-side such that resulting <temp3> has (8) columns altogether
        temp3 = pd.concat([temp1, temp2], axis=1)
        # Renaming columns of <temp3>
        temp3.columns = ['Star_ID1','RA_1', 'Dec_1', 'Mag_1', 'Star_ID2', 'RA_2', 'Dec_2', 'Mag_2']


        # Calculate angular distance between the two stars present in every row in <temp3>
        cols = ['RA_1', 'RA_2', 'Dec_1', 'Dec_2']
        temp3['Ang_Distance'] = temp3.apply(angularDistance, axis = 1, col_names = cols)

        # Generates <temp4> by selecting rows from <temp3> whose angular distances is
        # less than equal to the circular FOV limit
        temp4 = temp3[temp3.Ang_Distance <= FOV_limit]

        # Stores the value of the calculated mu_n for the current star
        mu_n_i = temp4.Ang_Distance.mean()
        
        # Multiply (mu_n_i) by (1/6) since sigma_i = Ang_distance_i, for all (i)
        mu_n_i = mu_n_i/6

        # Appends the entry to the SIGMA_CATALOGUE dataframe
        SIGMA_CATALOGUE = SIGMA_CATALOGUE.append({'Star_ID':star_id_i, 'mu_n':mu_n_i}, ignore_index=True)        
        
        # Releases memory back to OS
        if i%250 == 0:
            gc.collect()
            
            
    # Stop clock-1   
    end1 = time.time() - start1
    
    # Print time taken
    print('Time Taken - ', np.round(end1,3))
        
    return SIGMA_CATALOGUE

In [161]:
temp = genSigmaCatalogue(CATALOGUE, mag_limit=1, FOV_limit=20)

Number of stars -  16
Number of unique combinations per star=  240
Time Taken -  0.277


In [162]:
temp

,Star_ID,mu_n
0,23440.0,NaN
1,21936.0,NaN
2,50628.0,NaN
3,51926.0,1.669888
4,65795.0,NaN
5,17425.0,0.001373
6,17301.0,3.100984
7,27267.0,NaN
8,19984.0,3.100984
9,5297.0,NaN
